<a href="https://colab.research.google.com/github/MatiasSingermann/IBODS-AI/blob/main/IA_CLEAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importacion e instalacion de librerias

In [1]:
!pip install roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 38.2 MB/s 
     |████████████████████████████████| 178 kB 48.6 MB/s 
     |████████████████████████████████| 67 kB 4.4 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 138 kB 61.9 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Created wheel for roboflow: filename=roboflow-0.2.17-py3-none-any.whl size=31935 sha256=20325d6f6b37e1cf6bb0e61fdc09c2ed0e7c0f20c44591a0f998cf35f924e1c8
  Stored in directory: /root/.cache/pip/wheels/f2/38/3c/b4ac4d8a9d9b44bdcd51f6148ec810b0f05a404e5fed8df48d
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=8893bde797854c75b888ef32e65fcf0c033f2e673e38503a51c4cca946ad0c56
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built roboflow wget
  Attempting u

In [2]:
!pip install tflite-model-maker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 577 kB 17.3 MB/s 
     |████████████████████████████████| 238 kB 43.1 MB/s 
     |████████████████████████████████| 1.3 MB 42.4 MB/s 
     |████████████████████████████████| 1.1 MB 44.9 MB/s 
     |████████████████████████████████| 87 kB 7.3 MB/s 
     |████████████████████████████████| 840 kB 39.2 MB/s 
     |████████████████████████████████| 3.4 MB 17.6 MB/s 
     |████████████████████████████████| 60.2 MB 21 kB/s 
     |████████████████████████████████| 127 kB 19.0 MB/s 
     |████████████████████████████████| 10.9 MB 44.5 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 128 kB 40.1 MB/s 
     |████████████████████████████████| 25.3 MB 59.1 MB/s 
     |████████████████████████████████| 497.9 MB 28 kB/s 
     |████████████████████████████████| 352 kB 44.7 MB/s 
     |████████████████████████████

Importo `Tensorflow` y varios de sus **modulos**.

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tflite_model_maker import object_detector
import tensorflow_hub as hub
from tflite_model_maker.config import QuantizationConfig
from tflite_support.task import vision
from tflite_support.task.vision import ObjectDetector
from tflite_support.task.vision import ObjectDetectorOptions
from tflite_support.task.processor import DetectionOptions
from tflite_support.task.core import BaseOptions

Importo varias `librerias` que me permiten trabajar con **tensores**, y **operaciones matematicas** necesrias. 

In [4]:
import pandas as pd
import numpy as np
import torch
import torchvision.transforms as transforms

Importo `librerias` para **administrar archivos** y seleccionar de manera **random** alguno de ellos

In [5]:
import os
import random

Importo `librerias` que me permiten trabajar con las **imagenes** del dataset.

In [6]:
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from matplotlib.image import imread
from itertools import product
from skimage import draw, transform

Importo `Roboflow` para acceder al dataset

In [7]:
from roboflow import Roboflow

## Importacion del **Dataset** y sus **labels**

Me conecto a mi cuenta de Drive para agarrar ciertos archivos

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Importo el dataset desde Roboflow

In [9]:
rf = Roboflow(api_key="41l2d60Eq7cWhUOVgPxd")
project = rf.workspace("ibods").project("ibods_lbl")
dataset = project.version(1).download("voc")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to IBODS_LBL-1 in voc:: 100%|██████████| 7031/7031 [00:03<00:00, 2025.87it/s]


Asigno la **ruta del archivo** de las anotaciones a una variable

In [10]:
path_lbl = '/content/gdrive/MyDrive/Proyecto 4to/I.A/Anotaciones/labels.txt'

Abro el **archivo de anotaciones** y lo *recorro linea por linea*, *eliminando los espacios* y metiendo los `strings` en una `lista`

In [11]:
with open(path_lbl) as f:
   labels = [line.strip() for line in f.readlines()]
labels

['cordon',
 'autos',
 'personas',
 'cruces',
 'pozos',
 'parar',
 'cruzar',
 'bicicleta',
 'moto',
 'escalones']

Usando el **arhcivo de labels**, le asigno un *valor numerico* para que la `IA` pueda interpretar las anotaciones

In [12]:
assigned_labels = { i : labels[i] for i in range(0, len(labels) ) }
assigned_labels

{0: 'cordon',
 1: 'autos',
 2: 'personas',
 3: 'cruces',
 4: 'pozos',
 5: 'parar',
 6: 'cruzar',
 7: 'bicicleta',
 8: 'moto',
 9: 'escalones'}

## Funciones Utiles y configuracion

In [13]:
CLASSES = labels

In [14]:
COLORS = np.random.randint(0, 255, size=(len(CLASSES), 3), dtype=np.uint8)

In [15]:
def preprocess_image(image_path, input_size):
  ''' Preprocess the input image to feed to the TFLite model
  '''
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  return resized_img, original_image

In [16]:
def detect_objects(interpreter, image, threshold):
  ''' Returns a list of detection results, each a dictionary of object info.
  '''

  signature_fn = interpreter.get_signature_runner()

  # Feed the input image to the model
  output = signature_fn(images=image)

  # Get all outputs from the model
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results

In [17]:
def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  ''' Run object detection on the input image and draw the detection results
  '''
  
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path,
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(CLASSES[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

In [ ]:
def draw_bounding_box (image, model_path, threshold):
  base_opt = BaseOptions(model_path)

  detection_opt = DetectionOptions(score_threshold = threshold)

  options = ObjectDetectorOptions(base_opt, detection_opt)

  detector = ObjectDetector.create_from_options(options)

  tensor_image = vision.TensorImage.create_from_file(image)

  results = detector.detect(tensor_image)

  resultsdetect = results.detections

  for obj in resultsdetect:
    categories = obj.categories
    for category in categories:
      names = category.category_name
      scores = category.score * 100
    percentages = [float(scores)]
    for percentage in percentages:
      confidence = str(int(percentage)) + '%'
      label = names + ", " + confidence

  return label

## Cargar Dataset

### Cargo los datos de entrenamiento

In [ ]:
TRAIN_PATH = 'IBODS_LBL-1/train'

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    TRAIN_PATH,
    TRAIN_PATH,
    labels
)

### Cargo los datos de validacion

In [ ]:
VALID_PATH = 'IBODS_LBL-1/valid'

In [ ]:
valid_data = object_detector.DataLoader.from_pascal_voc(
    VALID_PATH,
    VALID_PATH,
    labels
)

### Cargo los datos de testeo

In [ ]:
TEST_PATH = 'IBODS_LBL-1/test'

In [ ]:
test_data = object_detector.DataLoader.from_pascal_voc(
    TEST_PATH,
    TEST_PATH,
    labels
)

## Entrenamiento de la IA

In [ ]:
model = object_detector.create(
    train_data,
    model_spec= 'efficientdet_lite1',
    batch_size= 8,
    train_whole_model=True,
    epochs=100,
    validation_data=valid_data
)

## Testeo de la IA y su performance

In [ ]:
model.evaluate(test_data)

## Salvo el modelo

In [18]:
os.mkdir('logs')

In [ ]:
model.export(export_dir='logs')

In [ ]:
model.evaluate_tflite('logs/model.tflite', test_data)

## Quantization

In [ ]:
config = QuantizationConfig.for_float16()

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('logs/model1.tflite')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()

OSError: ignored

## Deteccion

### Deteccion **simple** fotos

In [ ]:
base_opt = BaseOptions(
    'logs/model1.tflite'
)

In [ ]:
detection_opt = DetectionOptions(
    score_threshold = 0.3, 
)

In [ ]:
options = ObjectDetectorOptions(
    base_opt,
    detection_opt
)

In [ ]:
detector = ObjectDetector.create_from_options(
    options
)

In [ ]:
detector = vision.ObjectDetector.create_from_file('logs/model1.tflite')

In [ ]:
tensor_image = vision.TensorImage.create_from_file('IBODS_LBL-1/test/60_jpg.rf.af5b4d9b99351329bad5223344640506.jpg')

In [ ]:
detector.detect(tensor_image)

DetectionResult(detections=[Detection(bounding_box=BoundingBox(origin_x=63, origin_y=44, width=102, height=76), categories=[Category(index=1, score=0.9296875, display_name='', category_name='autos')]), Detection(bounding_box=BoundingBox(origin_x=43, origin_y=179, width=371, height=196), categories=[Category(index=4, score=0.85546875, display_name='', category_name='pozos')]), Detection(bounding_box=BoundingBox(origin_x=158, origin_y=61, width=67, height=55), categories=[Category(index=1, score=0.765625, display_name='', category_name='autos')]), Detection(bounding_box=BoundingBox(origin_x=240, origin_y=78, width=43, height=36), categories=[Category(index=1, score=0.6875, display_name='', category_name='autos')])])

In [ ]:
results = detector.detect(tensor_image)

In [ ]:
resultsdetect = results.detections

In [ ]:
resultsdetect[0]

Detection(bounding_box=BoundingBox(origin_x=63, origin_y=44, width=102, height=76), categories=[Category(index=1, score=0.9296875, display_name='', category_name='autos')])

In [ ]:
for obj in resultsdetect:
  box = obj.bounding_box
  print(box)

BoundingBox(origin_x=63, origin_y=44, width=102, height=76)
BoundingBox(origin_x=43, origin_y=179, width=371, height=196)
BoundingBox(origin_x=158, origin_y=61, width=67, height=55)
BoundingBox(origin_x=240, origin_y=78, width=43, height=36)


In [ ]:
for obj in resultsdetect:
  categories = obj.categories
  for category in categories:
    names = category.category_name
    scores = category.score * 100
  percentages = [float(scores)]
  for percentage in percentages:
    confidence = str(int(percentage)) + '%'
    label = names + ", " + confidence
    print(label)

autos, 92%
pozos, 85%
autos, 76%
autos, 68%


In [ ]:
for obj in resultsdetect:
  categories = obj.categories
  for category in categories:
    names = category.category_name
    print(names)

autos
pozos
autos
autos


In [ ]:
cwd = os.getcwd()

image_path = '145.jpg'
model_path = 'logs/model1.tflite'
threshold = 0.3

tensor_image = vision.TensorImage.create_from_file(image_path)
image = Image.open(image_path)
image = np.asarray(image)
wid = image.shape[1]
left = wid/3
center = left * 2
right = wid 

base_opt = BaseOptions(model_path)
detection_opt = DetectionOptions(score_threshold = threshold)
options = ObjectDetectorOptions(base_opt, detection_opt)

detector = ObjectDetector.create_from_options(options)

results = detector.detect(tensor_image)

resultsdetect = results.detections

_TEXT_COLOR = (0, 0, 255)

for obj in resultsdetect:
  print(obj.bounding_box.width, obj.bounding_box.height)
  start_point = obj.bounding_box.origin_x, obj.bounding_box.origin_y 
  end_point = obj.bounding_box.origin_x + obj.bounding_box.width, obj.bounding_box.origin_y + obj.bounding_box.height
  detect_img = cv2.rectangle(image, start_point, end_point, _TEXT_COLOR, 3)
  x1 = obj.bounding_box.origin_x
  x2 = obj.bounding_box.origin_x + obj.bounding_box.width
  middle = obj.bounding_box.width / 2
  Center = x2 - middle
  x = (x1, x2)
  if Center < left:
    position = "izquierda"
  elif Center < center:
    position = "centro"
  else:
    position = "derecha"
  categories = obj.categories
  for category in categories:
    names = category.category_name
    scores = category.score * 100
  percentages = [float(scores)]
  for percentage in percentages:
    confidence = str(int(percentage)) + '%'
    label = names + ", " + confidence + ", " + position
  cv2.putText(image, label, (obj.bounding_box.origin_x,obj.bounding_box.origin_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
  
img = Image.fromarray(detect_img, 'RGB')
img.save(f'{cwd}/result/output.png')
img.show()

212 408
421
40 118
145


In [45]:
def drawBoundingBox(image_path, model_path, threshold, classes):

  # Agarro el directorio (carpeta) actual
  cwd = os.getcwd()

  # Le asigno los tres parametros:
  # - La ruta de la imagen a detectar
  # - La ruta del modelo de IA
  # - El umbral de la IA

  image_path = image_path
  model_path = model_path
  threshold = threshold

  # Leo la imagen como un tensor
  tensor_image = vision.TensorImage.create_from_file(image_path)

  # Abro y leo la imagen como un array
  image = Image.open(image_path)
  image = np.asarray(image)

  # Divido la imagen en 3 (Izquierda, derecha y centro) utilizando el ancho de la imagen

  wid = image.shape[1]
  left = wid/3
  center = left * 2
  right = wid 

  # Asigno las opciones de configuracion del modelo de deteccion (Que modelo, el umbral, etc)
  base_opt = BaseOptions(model_path)
  detection_opt = DetectionOptions(score_threshold = threshold)
  options = ObjectDetectorOptions(base_opt, detection_opt)

  # Creo el modelo de deteccion
  detector = ObjectDetector.create_from_options(options)

  # Agarro los resultados de la deteccion
  results = detector.detect(tensor_image)
  resultsdetect = results.detections

  # Creo un randomizador de colores por clase
  CLASSES = classes
  COLORS = np.random.randint(0, 255, size=(len(CLASSES), 3), dtype=np.uint8)

  for obj in resultsdetect:
    categories = obj.categories
    for category in categories:
      names = category.category_name
      scores = category.score * 100
      class_id = category.index
    percentages = [float(scores)]
    color = [int(c) for c in COLORS[class_id]]
    start_point = obj.bounding_box.origin_x, obj.bounding_box.origin_y 
    end_point = obj.bounding_box.origin_x + obj.bounding_box.width, obj.bounding_box.origin_y + obj.bounding_box.height
    detect_img = cv2.rectangle(image, start_point, end_point, color, 2)
    x1 = obj.bounding_box.origin_x
    x2 = obj.bounding_box.origin_x + obj.bounding_box.width
    middle = obj.bounding_box.width / 2
    Center = x2 - middle
    x = (x1, x2)
    if Center < left:
      position = "izquierda"
    elif Center < center:
      position = "centro"
    else:
      position = "derecha"
    for percentage in percentages:
      confidence = str(int(percentage)) + '%'
      label = names + ", " + confidence
      cv2.putText(image, label, (obj.bounding_box.origin_x,obj.bounding_box.origin_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
      cv2.putText(image, position, (obj.bounding_box.origin_x,obj.bounding_box.origin_y - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
      
  img = Image.fromarray(detect_img, 'RGB')
  return img

In [46]:
img = drawBoundingBox('IBODS_LBL-1/test/60_jpg.rf.af5b4d9b99351329bad5223344640506.jpg','logs/model1.tflite', 0.3, labels)
img.save(f'{cwd}/result/output.png')

In [24]:
cwd = os.getcwd()
os.mkdir(f'{cwd}/result')

### Deteccion **compleja** fotos

In [ ]:
cwd = os.getcwd()
os.mkdir(f'{cwd}/result')

In [ ]:
cwd = os.getcwd()

DETECTION_THRESHOLD = 0.3

# Change the test file path to your test image
INPUT_IMAGE_PATH = 'IBODS_LBL-1/test/00--91-_png_jpg.rf.d3502e507f2bb074188f809e480b2fdb.jpg'

im = Image.open(INPUT_IMAGE_PATH)
im.thumbnail((512, 512), Image.ANTIALIAS)
im.save(f'{cwd}/result/input.png','PNG')

# Load the TFLite model
model_path = 'logs/model1.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
detection_result_image = run_odt_and_draw_results(
    INPUT_IMAGE_PATH,
    interpreter,
    threshold=DETECTION_THRESHOLD
)

# Show the detection result
img = Image.fromarray(detection_result_image)
img.save(f'{cwd}/result/output.png')

### Deteccion en video

In [ ]:
detector = vision.ObjectDetector.create_from_file('logs/model1.tflite')

import cv2

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("No se puede abrir la camara")
    exit()

while True:
  ret,frame = cap.read() # Voy agarrando los frames

  if ret == True: # Si leyo correctamente los frames, signfica que el retun (ret) es true, por lo que ejecuto el codigo en ese caso
    cv2.cvtColor(frame, cv2.COLOR_BGRRGB) # Convierto todos los frames al espectro RGB
    tensor_image = vision.TensorImage.create_from_array(frame) # Convierto todos los frames a tensores
    detector.detect(tensor_image)
    key = cv2.waitKey(1)
    if key == ord('q'):
      break
  if not ret:
        print("No puedo recibir imagenes (finalizaste la trasmision?). Saliendo ...")
        break
cap.release()
cv2.destroyAllWindows()

## Codigo Viejo o Extra

In [ ]:
def drawBox(image_path, model_path, threshold):

  image = cv2.imread(image_path)

  base_opt = BaseOptions(model_path)
  detection_opt = DetectionOptions(score_threshold = threshold)
  options = ObjectDetectorOptions(base_opt, detection_opt)

  detector = ObjectDetector.create_from_options(options)

  results = detector.detect(image)

  resultsdetect = results.detections

  _TEXT_COLOR = (0, 0, 255)

  for obj in resultsdetect:
    start_point = obj.bounding_box.origin_x, obj.bounding_box.origin_y 
    end_point = obj.bounding_box.origin_x + obj.bounding_box.width, obj.bounding_box.origin_y + obj.bounding_box.height
  detect_img = cv2.rectangle(image, start_point, end_point, _TEXT_COLOR, 3)
   
  return detect_img